In [1]:
#pip install pandas xlrd

Note: you may need to restart the kernel to use updated packages.


In [1]:
# configuration: show all columns
import pandas as pd
from functions import *

df_sharkattack= pd.read_excel('https://www.sharkattackfile.net/spreadsheets/GSAF5.xls')

Start of the cleaning process:
We remove the rows (variables) and columns (data according to date) that we will not need for the analysis of our project.
As this is a project to sell anti-shark attack bracelets, we do not consider it necessary to have data from past centuries, 
We need to have a trend of the data to be able to analyze it, for this reason and for having a considerable amount of data.
We have decided to keep the data from the 60's onwards, we could keep the data from 1900, but they do not provide a 
great value to the analysis and furthermore, they are quite incomplete.

In [2]:
df_sharkattack = clean_labels(df_sharkattack)
df_sharkattack = clean_rows(df_sharkattack)


Next, we will remove the columns that are not necessary for our analysis:

First, let's check and remove the columns that are made up of null values ​​in the vast majority.
Secondly, we are going to remove the columns from the variables type, name, injury, time, source that we do not need for our analysis as we have data that we do not see that influences sales

In [3]:
df_sharkattack = clean_country(df_sharkattack, 15)
df_sharkattack = clean_state(df_sharkattack, 12)
df_sharkattack = clean_sex(df_sharkattack, 'sex')

In [12]:
# show results
df_sharkattack.isnull().sum()

year          27
country       75
state        507
location     590
activity     611
sex          604
age         3019
death        587
dtype: int64

In [4]:
df_sharkattack.isnull().sum()

year           1
country        0
state          0
location      90
activity     351
sex          344
age         1793
death        389
dtype: int64